Scrape missing data from doesthedogdie.com

In [1]:
import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
import time

In [2]:
import sys
sys.path.append('../utils')
import functions

In [3]:
sample_df = pd.read_csv('../data/local/clean/movie_sample.csv')
# display(sample_df)

In [4]:
sample_df.head(5)

,title,clean_title,original_title,genres,director,release_year,runtime,budget,revenue,popularity,tmdb_rating,tmdb_votes,imdb_rating,imdb_votes,language,tmdb_id,imdb_id
0,Bad Boys for Life,bad boys for life,Bad Boys for Life,"thriller, action, crime","Bilall Fallah, Adil El Arbi",2020,124,90000000,426505244,67.7,7.1,8251,6.5,186716,English,38700,tt1502397
1,Avatar: The Way of Water,avatar the way of water,Avatar: The Way of Water,"science fiction, adventure, action",James Cameron,2022,192,460000000,2320250281,158.5,7.6,11946,7.5,518001,English,76600,tt1630029
2,Pet Sematary,pet sematary,Pet Sematary,"horror, thriller","Dennis Widmyer, Kevin Kölsch",2019,100,21000000,113118226,26.1,5.7,3186,5.7,100618,English,157433,tt0837563
3,How to Train Your Dragon: The Hidden World,how to train your dragon the hidden world,How to Train Your Dragon: The Hidden World,"animation, family, adventure",Dean DeBlois,2019,104,129000000,524580592,207.5,7.8,6381,7.4,158269,English,166428,tt2386490
4,Star Wars: The Rise of Skywalker,star wars the rise of skywalker,Star Wars: The Rise of Skywalker,"adventure, action, science fiction",J.J. Abrams,2019,142,416000000,1074144248,63.2,6.3,9886,6.4,507321,English,181812,tt2527338


Request for the first five rows in df

In [5]:
# # Load the API key from the .env file
# load_dotenv()
# API_KEY_DOESTHEDOGDIE = os.getenv("API_KEY_DOESTHEDOGDIE")

# # Sample data for demonstration purposes. Replace this with your actual DataFrame.
# # sample_df = pd.read_csv("path_to_your_csv.csv")  # Example of how you might load your DataFrame

# # Define the URL for the API request
# BASE_URL = "https://www.doesthedogdie.com/dddsearch?q="

# # Initialize a list to store the responses
# responses = []

# # Loop through the first 5 rows of 'clean_title' in the sample_df
# for clean_name in sample_df['clean_title'][:5]:  # First 5 rows
#     url = f"{BASE_URL}{clean_name}"
#     headers = {
#         "Accept": "application/json",
#         "X-API-KEY": API_KEY_DOESTHEDOGDIE,
#     }
    
#     # Make the GET request to the API
#     response = requests.get(url, headers=headers)
    
#     if response.status_code == 200:
#         responses.append(response.json())
#     else:
#         print(f"Error with request for {clean_name}: {response.status_code}")

# # Function to extract topics of interest from the 'stats' field
# def extract_topics(stats_str):
#     # Convert the string into a Python dictionary
#     try:
#         stats = json.loads(stats_str)  # stats is a string, so we parse it into a dictionary
#     except json.JSONDecodeError:
#         print("Error decoding stats:", stats_str)
#         return ""
    
#     # Check if 'topics' key exists
#     if 'topics' not in stats:
#         print("No 'topics' field in response:", stats)  # Print full response to investigate
#         return ""
    
#     # Extract topics where 'definitelyYes' > 'definitelyNo', and both are greater than 1
#     topics_of_interest = []
    
#     for topic_id, data in stats['topics'].items():
#         definitely_yes = int(data['definitelyYes'])  # Ensure definitelyYes is an integer
#         definitely_no = int(data['definitelyNo'])  # Ensure definitelyNo is an integer
        
#         # Apply condition: definitelyYes > definitelyNo and both values should be greater than 1
#         if definitely_yes > definitely_no:
#             topics_of_interest.append(str(topic_id))  # Store topic ID as a string
    
#     return ",".join(topics_of_interest)  # Join the topics into a comma-separated string

# # Add the 'topics' and 'doesthedog_id' columns to the DataFrame
# sample_df['topics'] = None
# sample_df['doesthedog_id'] = None

# # Loop through the responses and assign topics to the 'topics' column
# for index, response in enumerate(responses):
#     items = response.get('items', [])
#     if items:
#         stats_str = items[0].get('stats', '{}')  # Get the 'stats' field for the first item
#         topics = extract_topics(stats_str)
        
#         # Get the 'id' from the response (this is the 'id' field in each 'item')
#         doesthedog_id = items[0].get('id', None)
        
#         # Assign the topics and id to the correct row in the DataFrame
#         sample_df.loc[index, 'topics'] = topics
#         sample_df.loc[index, 'doesthedog_id'] = doesthedog_id

# # Display the DataFrame with the 'topics' and 'doesthedog_id' columns added (using 'clean_title' column)
# # print(sample_df[['clean_title', 'topics', 'doesthedog_id']])

Request in batches for the whole df

In [6]:
# # Load the API key from the .env file
# load_dotenv()
# API_KEY_DOESTHEDOGDIE = os.getenv("API_KEY_DOESTHEDOGDIE")

# # Define the URL for the API request
# BASE_URL = "https://www.doesthedogdie.com/dddsearch?q="

# # Function to extract topics of interest from the 'stats' field
# def extract_topics(stats_str):
#     # Convert the string into a Python dictionary
#     try:
#         stats = json.loads(stats_str)  # stats is a string, so we parse it into a dictionary
#     except json.JSONDecodeError:
#         print("Error decoding stats:", stats_str)
#         return ""
    
#     # Check if 'topics' key exists
#     if 'topics' not in stats:
#         print("No 'topics' field in response:", stats)  # Print full response to investigate
#         return ""
    
#     # Extract topics where 'definitelyYes' > 'definitelyNo', and both are greater than 1
#     topics_of_interest = []
    
#     for topic_id, data in stats['topics'].items():
#         definitely_yes = int(data['definitelyYes'])  # Ensure definitelyYes is an integer
#         definitely_no = int(data['definitelyNo'])  # Ensure definitelyNo is an integer
        
#         # Apply condition: definitelyYes > definitelyNo and both values should be greater than 1
#         if definitely_yes > definitely_no:
#             topics_of_interest.append(str(topic_id))  # Store topic ID as a string
    
#     return ",".join(topics_of_interest)  # Join the topics into a comma-separated string

# # Function to process the DataFrame in batches
# def process_in_batches(df, batch_size=50):
#     # Add the 'topics' and 'doesthedog_id' columns to the DataFrame
#     df['topics'] = None
#     df['doesthedog_id'] = None
    
#     # Initialize a list to store responses
#     responses = []
    
#     # Loop through the rows in batches
#     for start in range(0, len(df), batch_size):
#         end = min(start + batch_size, len(df))  # Define the batch end
#         batch = df.iloc[start:end]
        
#         # Initialize a list to store the current batch's responses
#         current_responses = []
        
#         for clean_name in batch['clean_title']:
#             url = f"{BASE_URL}{clean_name}"
#             headers = {
#                 "Accept": "application/json",
#                 "X-API-KEY": API_KEY_DOESTHEDOGDIE,
#             }
            
#             # Make the GET request to the API
#             response = requests.get(url, headers=headers)
            
#             if response.status_code == 200:
#                 current_responses.append(response.json())
#             else:
#                 print(f"Error with request for {clean_name}: {response.status_code}")
        
#         # Store the current batch responses
#         responses.extend(current_responses)
        
#         # Save backup after each batch to ensure progress is saved
#         df_backup = df.copy()
#         for index, response in enumerate(current_responses):
#             items = response.get('items', [])
#             if items:
#                 stats_str = items[0].get('stats', '{}')  # Get the 'stats' field for the first item
#                 topics = extract_topics(stats_str)
                
#                 # Get the 'id' from the response (this is the 'id' field in each 'item')
#                 doesthedog_id = items[0].get('id', None)
                
#                 # Assign the topics and id to the correct row in the DataFrame
#                 df_backup.loc[start + index, 'topics'] = topics
#                 df_backup.loc[start + index, 'doesthedog_id'] = doesthedog_id
        
#         # backup file 
#         df_backup.to_csv("backup_file.csv", index=False)
        
#         time.sleep(2)  # sleep time to prevent rate limiting
    
#     return df

# # Sample DataFrame (Replace with your actual DataFrame)
# # sample_df = pd.read_csv("path_to_your_csv.csv")  # Example of how you might load your DataFrame

# # Process the DataFrame in batches
# sample_df = process_in_batches(sample_df, batch_size=50)

# # Display the final DataFrame with 'topics' and 'doesthedog_id' columns added
# print(sample_df[['clean_title', 'topics', 'doesthedog_id']])


In [7]:
sample_df.head()

,title,clean_title,original_title,genres,director,release_year,runtime,budget,revenue,popularity,tmdb_rating,tmdb_votes,imdb_rating,imdb_votes,language,tmdb_id,imdb_id
0,Bad Boys for Life,bad boys for life,Bad Boys for Life,"thriller, action, crime","Bilall Fallah, Adil El Arbi",2020,124,90000000,426505244,67.7,7.1,8251,6.5,186716,English,38700,tt1502397
1,Avatar: The Way of Water,avatar the way of water,Avatar: The Way of Water,"science fiction, adventure, action",James Cameron,2022,192,460000000,2320250281,158.5,7.6,11946,7.5,518001,English,76600,tt1630029
2,Pet Sematary,pet sematary,Pet Sematary,"horror, thriller","Dennis Widmyer, Kevin Kölsch",2019,100,21000000,113118226,26.1,5.7,3186,5.7,100618,English,157433,tt0837563
3,How to Train Your Dragon: The Hidden World,how to train your dragon the hidden world,How to Train Your Dragon: The Hidden World,"animation, family, adventure",Dean DeBlois,2019,104,129000000,524580592,207.5,7.8,6381,7.4,158269,English,166428,tt2386490
4,Star Wars: The Rise of Skywalker,star wars the rise of skywalker,Star Wars: The Rise of Skywalker,"adventure, action, science fiction",J.J. Abrams,2019,142,416000000,1074144248,63.2,6.3,9886,6.4,507321,English,181812,tt2527338


Get film ids

In [8]:
# Load the API key from the .env file
load_dotenv()
API_KEY_DOESTHEDOGDIE = os.getenv("API_KEY_DOESTHEDOGDIE")

# Define the URL for the API request
BASE_URL = "https://www.doesthedogdie.com/dddsearch?q="

# Function to process the DataFrame in batches
def process_in_batches(df, batch_size=50):
    # Add the 'doesthedog_id' column to the DataFrame
    df['doesthedog_id'] = None
    
    # Initialize a list to store responses
    responses = []
    
    # Loop through the rows in batches
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))  # Define the batch end
        batch = df.iloc[start:end]
        
        # Initialize a list to store the current batch's responses
        current_responses = []
        
        for clean_name in batch['clean_title']:
            url = f"{BASE_URL}{clean_name}"
            headers = {
                "Accept": "application/json",
                "X-API-KEY": API_KEY_DOESTHEDOGDIE,
            }
            
            # Make the GET request to the API
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                current_responses.append(response.json())
            else:
                print(f"Error with request for {clean_name}: {response.status_code}")
        
        # Store the current batch responses
        responses.extend(current_responses)
        
        # Save backup after each batch to ensure progress is saved
        df_backup = df.copy()
        for index, response in enumerate(current_responses):
            items = response.get('items', [])
            if items:
                # Extract the 'id' from the response (this is the 'id' field in each 'item')
                doesthedog_id = items[0].get('id', None)
                
                # Assign the 'doesthedog_id' to the correct row in the DataFrame
                df_backup.loc[start + index, 'doesthedog_id'] = doesthedog_id
        
        # Save backup file 
        df_backup.to_csv("backup_file_id.csv", index=False)
        
        time.sleep(2)  # sleep time to prevent rate limiting
    
    return df

# Process the DataFrame in batches
sample_df = process_in_batches(sample_df, batch_size=50)

# Display the final DataFrame with 'doesthedog_id' column added
print(sample_df[['clean_title', 'doesthedog_id']])

KeyboardInterrupt: 